In [1]:
from data import *
import sys
import time
import math
import re
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

user = []
item = []
rating = []

d = Dataset()
d.load_items("allmovies.txt", item)
d.load_users("users.txt", user)
d.load_ratings("ratings.txt", rating)


n_users = len(user)
n_items = len(item)


In [3]:
utility = np.zeros((n_users, n_items))
for r in rating:
    utility[r.user_id - 1][r.item_id - 1] = r.rating

In [8]:
for i in utility:
    print(i)

[ 0.  0.  0. ...,  0.  0.  0.]
[ 9.  0.  0. ...,  0.  0.  0.]
[ 9.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]


In [11]:
movie_genre = []
for movie in item:
    movie_genre.append([movie.action, movie.adventure, movie.animation, movie.comedy,
                        movie.crime, movie.documentary, movie.drama, movie.family,
                        movie.fantasy, movie.history, movie.horror, movie.musical, 
                        movie.mystery, movie.romance, movie.sci_fi, movie.tv_movie, 
                        movie.thriller, movie.war, movie.western])

movie_genre = np.array(movie_genre)
cluster = KMeans(n_clusters=19)
((cluster.fit_predict(movie_genre)))

array([ 1,  4,  8, ..., 14, 11,  2], dtype=int32)

In [12]:
print(len(cluster.fit_predict(movie_genre)))

10131


In [17]:

utility_clustered = []
for i in range(0, n_users):
    average = np.zeros(19)
    # average = [[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 0,  0,  0,  0,  0,  0]]
    tmp = []
    for m in range(0, 19):
        tmp.append([])
        # tmp = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    for j in range(0, n_items):
        if utility[i][j] != 0:
            tmp[cluster.labels_[j] - 1].append(utility[i][j])
        # tmp = [[rate of movies belonging to cluster 0], [rate of movies belonging to cluster 1], ... [rate of movies belonging to cluster 18]]
    for m in range(0, 19):
        if len(tmp[m]) != 0:
            average[m] = np.mean(tmp[m])
        else:
            average[m] = 0
    utility_clustered.append(average)
utility_clustered = np.array(utility_clustered)

In [23]:
for i in range(0, n_users):
    x = utility_clustered[i]
    num = sum(a for a in x if a > 0)
    den = sum(a > 0 for a in x)
    if den == 0:
        user[i].avg_r = 0
    else:
        user[i].avg_r = num/den 
    print(user[i].avg_r)

0
5.125
7.06964285714
7.4


In [38]:
def pcs(x, y):
    num = 0
    den1 = 0
    den2 = 0
    A = utility_clustered[x - 1]
    B = utility_clustered[y - 1]
    num = sum((a - user[x - 1].avg_r) * (b - user[y - 1].avg_r) for a, b in zip(A, B) if a > 0 and b > 0)
    den1 = sum((a - user[x - 1].avg_r) ** 2 for a in A if a > 0)
    den2 = sum((b - user[y - 1].avg_r) ** 2 for b in B if b > 0)
    den = (den1 ** 0.5) * (den2 ** 0.5)
    if den == 0:
        return 0
    else:
        return num / den
pcs_matrix = np.zeros((n_users, n_users))
for i in range(0, n_users):
    for j in range(0, n_users):
        if i!=j:
            pcs_matrix[i][j] = pcs(i + 1, j + 1)


In [39]:
pcs_matrix

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.62014004, -0.46598257],
       [ 0.        ,  0.62014004,  0.        , -0.32321283],
       [ 0.        , -0.46598257, -0.32321283,  0.        ]])